<h2>Active User Retention</h2>
<p>Difficulty: Hard</p>
<i><b>Question: </b>Assume you're given a table containing information on Facebook user actions. Write a query to obtain number of monthly active users (MAUs) in July 2022, including the month in numerical format "1, 2, 3".

Hint:

An active user is defined as a user who has performed actions such as 'sign-in', 'like', or 'comment' in both the current month and the previous month.</i>

[Link to the question](https://datalemur.com/questions/user-retention)

In [ ]:
with sort_table as(
  SELECT 
    *,
    EXTRACT(month FROM event_date) as mnth,
    CASE WHEN 
            EXTRACT(month FROM event_date) - 1 = 
            LAG(EXTRACT(month FROM event_date), 1) OVER(ORDER BY user_id) 
              then 1 
              else 0
          END as sort
  FROM user_actions
  WHERE
    EXTRACT(month FROM event_date) IN (6,7) AND
    EXTRACT(year FROM event_date) = 2022 AND
    event_type IN ('sign-in', 'like', 'comment')
)

SELECT 
  mnth,
  COUNT(mnth) as monthly_active_users
FROM sort_table
WHERE
  sort = 1
GROUP BY
  mnth

<h2>Highest-Grossing Items</h2>
<p>Difficulty: Medium</p>
<i><b>Question: </b>Assume you're given a table containing data on Amazon customers and their spending on products in different category, write a query to identify the top two highest-grossing products within each category in the year 2022. The output should include the category, product, and total spend.</i>

[Link to the question](https://datalemur.com/questions/sql-highest-grossing)

In [ ]:
SELECT 
  category,
  product,
  total_spend
FROM (
    -- assigning a rank to the total sum of the product 
    -- in the category from the largest to the smallest 
    -- with an annual filter
  SELECT 
    category,
    product,
    SUM(spend) as total_spend,
    RANK() OVER(
      PARTITION BY category
      ORDER BY SUM(spend) desc) as rank_id
  FROM product_spend
  WHERE
    EXTRACT(year FROM transaction_date) = 2022
  GROUP BY
    category,
    product
  ORDER BY
    category) as rank_table 

WHERE
  rank_id IN (1,2)

<h2>User's Third Transaction</h2>
<p>Difficulty: Medium</p>
<i><b>Question: </b>Assume you are given the table below on Uber transactions made by users. Write a query to obtain the third transaction of every user. Output the user id, spend and transaction date.</i>

[Link to the question](https://datalemur.com/questions/sql-third-transaction)

In [ ]:
SELECT
  user_id,
  spend,
  transaction_date
FROM (
  SELECT
    *,
    RANK()
      OVER(
        PARTITION BY user_id 
        ORDER BY transaction_date) as rank_id
  FROM transactions ) as filter_tabel
WHERE 
    rank_id =3

<h2>Cards Issued Difference</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Your team at JPMorgan Chase is preparing to launch a new credit card, and to gain some insights, you're analyzing how many credit cards were issued each month.

Write a query that outputs the name of each credit card and the difference in the number of issued cards between the month with the highest issuance cards and the lowest issuance. Arrange the results based on the largest disparity.</i>

 [Link to the question](https://datalemur.com/questions/cards-issued-difference)

In [ ]:
SELECT 
  card_name,
  MAX(issued_amount) - MIN(issued_amount) as sum
FROM monthly_cards_issued
GROUP BY
  card_name
ORDER BY
  sum DESC

<h2>Duplicate Job Listings</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Assume you're given a table containing job postings from various companies on the LinkedIn platform.
 Write a query to retrieve the count of companies that have posted duplicate job listings.</i>

 [Link to the question](https://datalemur.com/questions/duplicate-job-listings)

In [ ]:
SELECT 
    COUNT(*)
FROM (
    SELECT company_id
    FROM job_listings
    GROUP BY company_id
    HAVING count(company_id) > 1
    ) as filter_table

<h2>Teams Power Users</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Write a query to identify the top 2 Power Users who sent the highest number of messages on Microsoft Teams in August 2022. Display the IDs of these 2 users along with the total number of messages they sent. Output the results in descending order based on the count of the messages.</i>

[Link to the question](https://datalemur.com/questions/teams-power-users)

In [ ]:
SELECT 
  sender_id,
  COUNT(sender_id) as message_namber
FROM messages
WHERE
  sent_date BETWEEN '2022-08-01' AND '2022-08-31'
GROUP BY
  sender_id
ORDER BY
  message_namber DESC
LIMIT 2

<h2>Average Review Ratings</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Given the reviews table, write a query to retrieve the average star rating for each product, grouped by month. The output should display the month as a numerical value, product ID, and average star rating rounded to two decimal places. Sort the output first by month and then by product ID.</i>

[Link to the question](https://datalemur.com/questions/sql-avg-review-ratings)

In [ ]:
SELECT 
  EXTRACT(MONTH FROM submit_date) as Month,
  product_id,
  round(AVG(stars), 2) as AVG_Rate_Prod
FROM  reviews
GROUP BY
  Month, 
  product_id
ORDER BY
  Month, 
  product_id

<h2>Page With No Likes</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Assume you're given two tables containing data about Facebook Pages and their respective likes (as in "Like a Facebook Page").

Write a query to return the IDs of the Facebook pages that have zero likes. The output should be sorted in ascending order based on the page IDs.</i>

[Link to the question](https://datalemur.com/questions/sql-page-with-no-likes)

In [ ]:
SELECT 
  p.page_id as page
FROM 
  pages as p
FULL OUTER JOIN page_likes 
  ON p.page_id = page_likes.page_id
GROUP BY
  page
HAVING
  COUNT(page_likes.liked_date) = 0
ORDER BY
  page ASC

<h2>App Click-through Rate (CTR)</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Assume you have an events table on Facebook app analytics. Write a query to calculate the click-through rate (CTR) for the app in 2022 and round the results to 2 decimal places.

Definition and note:

Percentage of click-through rate (CTR) = 100.0 * Number of clicks / Number of impressions
To avoid integer division, multiply the CTR by 100.0, not 100.</i>

[Link to the question](https://datalemur.com/questions/click-through-rate)

In [ ]:
SELECT
  app_id,
  round(
    100.0 * 
    SUM (CASE WHEN event_type = 'click' THEN 1 END) / 
    SUM (CASE WHEN event_type = 'impression' THEN 1  END), 2
  ) as ctr
FROM events
WHERE
  EXTRACT(Year FROM timestamp) = '2022'
GROUP BY
  app_id

<h2>Cities With Completed Trades</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Assume you're given the tables containing completed trade orders and user details in a Robinhood trading system.

Write a query to retrieve the top three cities that have the highest number of completed trade orders listed in descending order. Output the city name and the corresponding number of completed trade orders.</i>

[Link to the question](https://datalemur.com/questions/completed-trades)

In [ ]:
SELECT 
  city,
  COUNT(*) as total_orders
FROM trades
LEFT JOIN users 
  ON trades.user_id = users.user_id
GROUP BY
  status,
  city
HAVING
    status ='Completed'
ORDER BY
  total_orders DESC
LIMIT 3

<h2>Second Day Confirmation</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Assume you're given tables with information about TikTok user sign-ups and confirmations through email and text. New users on TikTok sign up using their email addresses, and upon sign-up, each user receives a text message confirmation to activate their account.

Write a query to display the user IDs of those who did not confirm their sign-up on the first day, but confirmed on the second day.

Definition:

action_date refers to the date when users activated their accounts and confirmed their sign-up through text messages.</i>

[Link to the question](https://datalemur.com/questions/second-day-confirmation)

In [ ]:
SELECT 
  user_id
FROM emails
RIGHT JOIN texts 
  ON emails.email_id = texts.email_id
WHERE
  signup_action = 'Confirmed' AND
  EXTRACT(day FROM (action_date - signup_date)) = 1